# Create a more complex ai assistant 
- This example we create an ai assistant and ask it to evaluate some requirements.

In [1]:
from openai import OpenAI
import os

In [2]:
# Build veriables
API_KEY=os.environ.get('OPENAI_API_KEY')
ASSISTANT_NAME="SysML_Req_Eval"
INSTRUCTION_TEXT = """You are a helpful assistant. Given a set of sysml model files, you understand the paramters, calculations, and requirments and evaluate the requirments."""
CONTENT_TEXT = "Please evaluate the model requirments and summarize the requirments evaluation results in a table with the colums ID, Subject, Description, Actual Value, Calculated Value, Result (pass.fail)"

In [3]:
# Set up directories
BASE_DIR = os.getcwd()
RAG_FILES_DIR = "models"
WORKING_DIR = ""
MODEL_NAME = "block_volume.sysml"
FILE_PATH = f'{BASE_DIR}/{WORKING_DIR}/{RAG_FILES_DIR}/{MODEL_NAME}' 
FILE_PATH

'/usr/src/app/notebooks/SysML_Testing/volume_calc//models/block_volume.sysml'

In [4]:
%put file_content
# Python kernel cell
# Read the file and store its content in a variable
file_path = FILE_PATH
with open(file_path, 'r') as file:
    file_content = file.read()
    print(file_content)

package example {
    import ScalarValues::*;
    import SI::*;
    
    doc /* LLM Integration example: 
    Here we create a block 
    1. with dimenstions (length, width, and height) and Volume.
    2. We create and define the calculation for the volume.
    3. We define the blocks part usage of m_volume to be the returned calculated volume.
    
    To integrate with an LLM lets perform the following steps:
    A. Create an exmaple Scilab API
    B. Create a Analysis Template which describes how to use the API
    C. Build a LLM Assitant and ask the assistance to reutrn code to perform the analysis using the models and the API. 
    */
    
    attribute def Length;
    attribute def Width;
    attribute def Height;
    attribute def Volume;
    
    // Definition of a block
    part def Block {
        attribute length: Length:Real;
        attribute width: Width:Real;
        attribute height: Height:Real;
        attribute volume: Volume:Real;
        
        // Calulcation def

In [5]:
client = OpenAI(api_key=API_KEY,)
client

In [6]:
file_ids = []
# Assuming client.files.create correctly uploads the files and returns their file objects
model = client.files.create(file=open(FILE_PATH, "rb"), purpose='assistants')
file_ids.append(model.id)
print(file_ids)

['file-KPzb3U5WEhLEykNtaQlYIsjO']


In [7]:
tools = [{"type": "retrieval"}]
tools

[{'type': 'retrieval'}]

In [8]:
# Pass file_ids directly to the assistant creation call
# Note we cannot set temperature or top_p for python right now
assistant = client.beta.assistants.create(
    name=ASSISTANT_NAME,
    instructions=INSTRUCTION_TEXT,
    model="gpt-4-1106-preview",
    tools=tools,  # Ensure tools is defined earlier or include the necessary tools here
    file_ids=file_ids  # Use the list directly without wrapping it in another list

)

assistant_id = assistant.id
print(assistant_id)

asst_hgz5RLSoHAI2GfxYseGeBXqp


In [9]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": CONTENT_TEXT,
      "file_ids": file_ids
    }
  ]
)

thread_id = thread.id
thread_id

'thread_dqrt9mNVue4hkmE6HUQlr10n'

In [10]:
# Create the thread
run = client.beta.threads.runs.create(
  thread_id=thread_id,
  assistant_id=assistant_id
)

run_id = run.id
run_id

'run_gaVO05RbKd1KCwuZPNQ0LZuS'

In [11]:
import time

# Start an infinite loop to continuously check the status
while True:
    # Retrieve the current status of the run
    run_status_response = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
    
    # Check if the response has the attribute or key for status; adjust based on the response structure
    if hasattr(run_status_response, 'status'):  # If the response is an object with attributes
        current_status = run_status_response.status
    elif 'status' in run_status_response:  # If the response is a dictionary
        current_status = run_status_response['status']
    else:
        print("Unable to find status in the response.")
        break
    
    print(f"Current status: {current_status}")
    
    # Break the loop if the run status is 'complete' or adjust according to the correct completion status
    if current_status == 'completed':
        print("Run is complete.")
        break
    elif current_status == 'failed':
        print("Run failed.")
        break
    
    # Sleep for a bit before checking the status again to avoid overwhelming the server
    time.sleep(2)

Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: completed
Run is complete.


In [12]:
# Retrieve the list of messages for the specified thread_id
thread_messages = client.beta.threads.messages.list(thread_id=thread_id)
thread_messages

SyncCursorPage[Message](data=[Message(id='msg_wHEOHKezwZ6RNcC81Qi7Jbo0', assistant_id='asst_hgz5RLSoHAI2GfxYseGeBXqp', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Based on the SysML model file information provided, I will now summarize the requirements evaluation results in the requested table format, which includes columns for ID, Subject, Description, Actual Value, Calculated Value, and Result (pass/fail). We will calculate this based on the given dimensions for `block_1` and the constraints provided in the requirements definitions.\n\nFirst, we need to calculate the volume using the formula provided in the `CalculatedBlockVolume` calculation, which indicates that the block's volume is calculated by multiplying the length (l), width (w), and height (h+5[m]) attributes of `block_1`.\n\nGiven Dimensions for `block_1`:\n- Length: 2[m]\n- Width: 2[m]\n- Height: 2[m]\n\nUsing the formula, the Calculated Volume is \\(2 \\times 2 \\times (2 + 5) = 28 [m^3]\

In [13]:
# Function to handle and print each content block in a message
def print_content_blocks(content_blocks):
    for block in content_blocks:
        # Assuming 'type' and 'text' are attributes of the block object
        if hasattr(block, 'type') and block.type == 'text':
            # Further assuming 'value' can be accessed directly from the 'text' attribute
            if hasattr(block.text, 'value'):
                text_value = block.text.value
                print(f"Content: {text_value}")
            else:
                print("Content: [Text block without accessible value]")
        else:
            # Handle other types of content blocks or print a placeholder
            print("Content: [Non-text content block]")

# Sample code to iterate through messages and print their content
for message in thread_messages.data:
    print(f"\nRole: {message.role}")
    
    # If 'content' is expected to be a list or similar iterable of content blocks
    if hasattr(message, 'content') and isinstance(message.content, list):
        print_content_blocks(message.content)
    else:
        # If 'content' is not a list, try to print it directly (or handle it as needed)
        print("Content: [Message content is not a list]")
    
    # Check for attached files
    if hasattr(message, 'file_ids') and message.file_ids:
        print("Attached Files:")
        for file_id in message.file_ids:
            print(f" - {file_id}")
    else:
        print("No attached files.")

    print("\n" + "-"*50)



Role: assistant
Content: Based on the SysML model file information provided, I will now summarize the requirements evaluation results in the requested table format, which includes columns for ID, Subject, Description, Actual Value, Calculated Value, and Result (pass/fail). We will calculate this based on the given dimensions for `block_1` and the constraints provided in the requirements definitions.

First, we need to calculate the volume using the formula provided in the `CalculatedBlockVolume` calculation, which indicates that the block's volume is calculated by multiplying the length (l), width (w), and height (h+5[m]) attributes of `block_1`.

Given Dimensions for `block_1`:
- Length: 2[m]
- Width: 2[m]
- Height: 2[m]

Using the formula, the Calculated Volume is \(2 \times 2 \times (2 + 5) = 28 [m^3]\).

Now let us evaluate against the defined requirements:

1. `maxBlockVolume` (Maximum Volume Requirement)
   - Subject: `block_1`
   - Actual Value: \(5.5 [m^3]\)
   - Calculated Va

In [14]:
import re

# Function to extract and print Python code blocks from the raw message data
def extract_python_code(raw_messages):
    python_code_blocks = []
    
    # Regular expression pattern to match Python code blocks
    python_code_pattern = re.compile(r"```python\n(.*?)```", re.DOTALL)
    
    for message in raw_messages:
        for content_block in message.content:
            # Assuming each content_block has a 'text' object with a 'value' attribute
            text_content = content_block.text.value
            
            # Search for Python code blocks within the text content
            matches = python_code_pattern.findall(text_content)
            python_code_blocks.extend(matches)
    
    return python_code_blocks

# Assuming 'thread_messages.data' is a list of messages as described in your raw thread message structure
raw_messages = thread_messages.data  # This should be the actual data structure containing your messages

# Extract Python code blocks
extracted_python_code = extract_python_code(raw_messages)

# Print extracted Python code blocks
for index, code in enumerate(extracted_python_code, start=1):
    print(f"Python Code Block {index}:\n{code}\n\n---\n")


In [15]:
# Lets clean up the assistnat and the files

# Delete the files first
client.files.delete(file_ids[0])

# Then delete the assistant
client.beta.assistants.delete(assistant_id)

AssistantDeleted(id='asst_hgz5RLSoHAI2GfxYseGeBXqp', deleted=True, object='assistant.deleted')